In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series
#ロジスティック回帰による二項分類器

In [2]:
#一行目は、乱数のシード(種）を指定するもでである。
#ここで指定した値によって、その後に生成される乱数のパターンが決まる。
np.random.seed(20160512)

#t=0（非感染）のデータを乱数で発生させる。
n0, mu0, variance0 = 20, [10, 11], 20
#np.eyeは単位行列、multivariate_normal(mean, cov[, size])
data0 = multivariate_normal(mu0, np.eye(2)*variance0, n0)
df0 = DataFrame(data0, columns=['x1','x2'])
df0['t'] = 0

#t=1（感染）のデータを乱数で発生させる。
n1, mu1, variance1 = 15, [18, 20], 22
data1 = multivariate_normal(mu1, np.eye(2)*variance1 ,n1)
df1 = DataFrame(data1, columns=['x1','x2'])
df1['t'] = 1

#これらのデータを一つにまとめた上で、より本物のデータらしくするために、行の順番をランダムにに入れ替えている。
df = pd.concat([df0, df1], ignore_index=True)
train_set = df.reindex(permutation(df.index)).reset_index(drop=True)

In [3]:
train_set

,x1,x2,t
0,20.729880,18.209359,1
1,16.503919,14.685085,0
2,5.508661,17.426775,0
3,9.167047,9.178837,0
4,8.640423,9.561952,0
5,10.500988,17.002584,0
6,16.484766,22.232997,1
7,6.979059,8.180596,0
8,11.701339,0.996734,0
9,21.367990,18.712309,1


In [4]:
train_x = train_set[['x1','x2']].as_matrix()
train_t = train_set['t'].as_matrix().reshape([len(train_set), 1])

In [5]:
#ステップ1_与えられたデータを元にして、未知のデータを予測する数式を考える
x = tf.placeholder(tf.float32, [None, 2])
w = tf.Variable(tf.zeros([2, 1]))
#一要素の一次元リストとして定義
w0 = tf.Variable(tf.zeros([1]))
f = tf.matmul(x, w) + w0
#fをシグモイド関数
p = tf.sigmoid(f)

In [6]:
#ステップ2_数式に含まれるパラメーターの良し悪しを判断する誤差関数を用意する。
t = tf.placeholder(tf.float32, [None, 1])
#誤差関数の設定
loss = -tf.reduce_sum(t*tf.log(p) + (1-t)*tf.log(1-p))
#ステップ3、誤差関数を最小にするようにパラメータの値を決定する。
#トレーニングアルゴリズムの指定
train_step = tf.train.AdamOptimizer().minimize(loss)

In [7]:
#（Pn-0.5）と（tn-0.5）の符号を比較することで、予測が正解しているかどうかを判定している。
#tf.signは符号を取り出す関数で、tf.eqaulは2つの引数の値が等しいかどうかを判定して、Bool値買えす関数である。
correct_prediction = tf.equal(tf.sign(p-0.5), tf.sign(t-0.5))
#tf.cast関数でBool値を1.0の値に変換して、全体の平均値を計算している。
#tf.reduce_meanは、ベクトルの各成分の平均値を計算する関数である。
#正解なら1、不正解なら0が並んだベクトルの平均値を計算し、これが結局、正解率の値となる。
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
#セッションの用意
sess = tf.Session()
#Variableの値の初期化
sess.run(tf.initialize_all_variables())

In [9]:
#ステップ3_誤差関数を最小にするようにパラメーターの値を決定する。
#勾配降下法によるパラメータの最適化を20000回繰り返します。
#ここでは、2000回繰り返すごとに、その時点での誤差関数lossと正解率accuracyの値を計算して表示している。
i = 0
for _ in range(20000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 2000 == 0:
        loss_val, acc_val = sess.run(
            [loss, accuracy], feed_dict={x:train_x, t:train_t})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

Step: 2000, Loss: 15.165894, Accuracy: 0.885714
Step: 4000, Loss: 10.772635, Accuracy: 0.914286
Step: 6000, Loss: 8.197757, Accuracy: 0.971429
Step: 8000, Loss: 6.576121, Accuracy: 0.971429
Step: 10000, Loss: 5.511984, Accuracy: 0.942857
Step: 12000, Loss: 4.798029, Accuracy: 0.942857
Step: 14000, Loss: 4.314191, Accuracy: 0.942857
Step: 16000, Loss: 3.986273, Accuracy: 0.942857
Step: 18000, Loss: 3.766507, Accuracy: 0.942857
Step: 20000, Loss: 3.623062, Accuracy: 0.942857


In [10]:
#最適化の処理はここで打ち切って、この時点でのパラメーター（Variable）の値を取得している。
#w0とwは、それぞれ、一要素のみのリスト、及び、2×1行列として定義されている。
w0_val, w_val = sess.run([w0, w])
#インデックスを指定して具体的な値の部分を取り出す。
w0_val, w1_val, w2_val = w0_val[0], w_val[0][0], w_val[1][0]
print w0_val, w1_val, w2_val

-15.6304 0.560301 0.492597


In [12]:
train_set0 = train_set[train_set['t']==0]
train_set1 = train_set[train_set['t']==1]

fig = plt.figure(figsize=(6,6))
subplot = fig.add_subplot(1,1,1)
subplot.set_ylim([0,30])
subplot.set_xlim([0,30])
subplot.scatter(train_set1.x1, train_set1.x2, marker='x')
subplot.scatter(train_set0.x1, train_set0.x2, marker='o')

linex = np.linspace(0,30,10)
liney = - (w1_val*linex/w2_val + w0_val/w2_val)
subpelot.plot(linex, liney)
field = [[(1 / (1 + np.exp(-(w0_val + w1_val*x1 + w2_val*x2))))
         for x1 in np.linspace(0,30,100)]
        for x2 in np.linspace(0,30,100)]
subplot.imshow(field, origin='lower', extent=(0,30,0,30),
              cmap=plt.cm.gray_r, alpha=0.5)
plt.show()

/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':
